This notebook is used to determine which strategy is the best for any particular asset.

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join("..", "src"))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd

df = pd.read_csv("../data/processed/starting_portfolio.csv")

In [ ]:
import yfinance as yf

start_date = "2024-03-01"
end_date = "2025-02-28"

portfolio = {}
for asset, weight in list(map(list, df.values)):
    portfolio[asset] = {
        "data": yf.Ticker(asset).history(start=start_date, end=end_date, actions=False),
        "weight": weight,
        "strategy": None,
        "best_return": float("-inf"),
    }

## Testing with one asset and one strategy

In [ ]:
from backtesting import Backtest

from strategies.larry_williams_price_action import LarryWilliamsPriceAction

data = portfolio["BXP"]["data"]
data.index = data.index.values.astype("datetime64[D]")
bt = Backtest(
    data, LarryWilliamsPriceAction, cash=1000000, commission=0.002, exclusive_orders=True
)
stats = bt.run()
return_pct = stats["Return [%]"]
if return_pct >= portfolio["BXP"]["best_return"]:
    portfolio["BXP"]["strategy"] = bt._strategy.__name__
    portfolio["BXP"]["best_return"] = return_pct

In [ ]:
stats._trades

## All assets and all strategies

In [ ]:
from backtesting import Backtest

from strategies.larry_williams_price_action import LarryWilliamsPriceAction
from strategies.macd_bollinger_bands_mean_reversion import MACDBollingerBandsMeanReversion
from strategies.bollinger_bands_breakout import BollingerBandsBreakout

# TODO: Add more
strategies = [LarryWilliamsPriceAction, MACDBollingerBandsMeanReversion, BollingerBandsBreakout]

for asset in portfolio:
    for strategy in strategies:
        data = portfolio[asset]["data"]
        data.index = data.index.values.astype("datetime64[D]")
        bt = Backtest(
            data, strategy, cash=1000000, commission=0.002, exclusive_orders=True
        )
        stats = bt.run()
        return_pct = stats["Return [%]"]
        if return_pct >= portfolio[asset]["best_return"]:
            portfolio[asset]["strategy"] = bt._strategy.__name__
            portfolio[asset]["best_return"] = return_pct

In [ ]:
results = pd.DataFrame(
    [
        [
            asset,
            portfolio[asset]["weight"],
            portfolio[asset]["strategy"],
        ]
        for asset in portfolio
    ],
    columns=["asset", "weight", "strategy"]
)

results.to_csv("../data/misc/example_backtesting.csv", index=False)